In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
from IPython.display import clear_output
import socket
import time
from datetime import datetime
import shutil
from random import randint
import re
import imagecodecs
import tifffile
import tensorflow as tf
from scipy.spatial import Voronoi, cKDTree
from matplotlib.path import Path
from rtree import index
from ctypes.wintypes import INT
import time
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import torch, torchvision
import colorsys

print("torchvision: ", torchvision.__version__)
print('CUDA available:', torch.cuda.is_available())

In [ ]:
def get_color_by_id(point_region_id, total_ids):
    hue = point_region_id / total_ids # Scale the hue by the number of unique IDs, wrapping around the hue circle
    saturation = 0.9; value = 0.9  # Keep saturation and value high for bright colors
    rgb = colorsys.hsv_to_rgb(hue, saturation, value)
    r, g, b = (int(c * 255) for c in rgb)  # Convert to 0-255 scale for RGB colors
    return (r << 16) | (g << 8) | b  # Combine into a single integer

def int24_to_rgb(objID):
    r = (objID >> 16) & 0xFF  # Extract the highest 8 bits
    g = (objID >> 8) & 0xFF   # Extract the middle 8 bits
    b = objID & 0xFF          # Extract the lowest 8 bits
    return r, g, b

def ellipse_points(cx, cy, rx, ry, n=8):
    """Generate n points along the ellipse centered at (cx, cy) with radii rx and ry."""
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    points = [(cx + rx * np.cos(angle), cy + ry * np.sin(angle)) for angle in angles]
    return points

def draw_polygons(draw, polygons):
    global globalID
    for polygon in polygons:
        if len(polygon) < 2:
            continue
        # color = get_color_by_id(i, len(polygons))
        # draw.polygon(polygon, fill=i, outline='white')
        globalID+=1
        # there still aren't unique identifiers for each instance of a polygon
        #   I wonder if this is because the conversion from 24 bit to 8 bit integers doesn't guarantee a unique set of 8 bit integers for each 24 bit integer 
        (r,g,b) = int24_to_rgb(globalID)
        if (globalID == 300): print(globalID,r,g,b)
        draw.polygon(polygon, fill=(r,g,b))

In [ ]:
yolo_model_path = r"S:\Phys\FIV925 XSection\Datasets\Axons vs SCs\01i5\YO 384 0530 MAXI\map75=0855192 YOLOv8n-seg map75=081193 idx=4 ep=24 btch=32 rnd=2840511\weights\best.pt"
model = YOLO(yolo_model_path)
pred_height, pred_width = 384, 384
test_rows = 2
test_cols = 2
iteration = 0
scale_factor = 1
globalID = 0
maxArea = 10000
dataset = "Human Lung Cancer"
base_path = r"R:\FIVE\EXP\FIV925\Additional Datasets"
ome_tiff_path = f"{base_path}\\{dataset}\\morphology_mip.ome.tif"
use_entire_image = True

image = tifffile.TiffFile(ome_tiff_path).asarray()
image_data = image[::2, ::2]
max_val_16bit = np.max(image_data)
image_scaled = (image_data / max_val_16bit) * 255
image_8bit = image_scaled.astype(np.uint8)

# create image and resize
og_image = Image.fromarray(image_8bit, 'L')
new_width = int(og_image.width * scale_factor); new_height = int(og_image.height * scale_factor)
image_resized = og_image.resize((new_width, new_height), Image.LANCZOS)
image = np.array(image_resized)

start_y = pred_height * 10
start_x = pred_width * 10
image_height_use = pred_height * test_rows
image_width_use = pred_width * test_cols

if use_entire_image:
    start_y = 0
    start_x = 0
    image_height_use = image_resized.height
    image_width_use = image_resized.width
    

big_array = np.zeros((image_height_use, image_width_use)).astype(np.int32)
list_xywh = []
maxValue = 0

# as we're making predictions, we want a big array that's meant to 'contain' the whole image
#   then insert the predictions into that big array
# so here is where we go through the image as tiles and make the predictions
tile_id = 0

print("height of image we're predicting on:", image_height_use)
print("width of image we're predicting on:", image_width_use)
print("height of image:", image.shape[0])
print("width of image:", image.shape[1])
print("big_array height:", big_array.shape[0])
print("big_array width:", big_array.shape[1])

In [ ]:
for i in range(start_x, image_width_use + pred_width, pred_width): # section_minx, section_maxx
    for j in range(start_y, image_height_use + pred_height, pred_height): # section_miny, section_maxy
        iteration += 1
        tile = image[j:j+pred_height, i:i+pred_width]
        
        print("x location in image:", i)
        print("y location in image:", j)
        
        print("tile height:", tile.shape[0])
        print("tile width:", tile.shape[1])
        
        tile_id += 1
        
        if tile.shape[0] == 0 or tile.shape[1] == 0:
            continue  # Skip tiles that have no width or height
        tile_as_img = Image.fromarray(tile, 'L')
        predictions = model.predict(tile_as_img, show=False, max_det=3000) # can we pass an np array?
        
        result = predictions[0]
        # draw masks onto blank image
        if result.masks is not None:
            polygons = result.masks.xy
        else:
            if result.boxes is not None:
                ## Ellipse Version
                polygons = []
                for (cx, cy, w, h) in result.boxes.xywh:
                    rx = w / 2
                    ry = h / 2
                    ellipse = ellipse_points(cx, cy, rx, ry, 8)
                    if (w*h)>maxArea: continue
                    polygons.append(ellipse)
                
                ## Bounding Box Version
                # xyxy = result.boxes.xyxy
                # for (x1, y1, x2, y2) in xyxy:
                #     polygon = [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
                #     polygons.append(polygon)
            else:
                print("No Polygons or Boxes"); continue
        
        blank_tile = np.zeros((pred_width, pred_height,3))
        blank_tile_as_img = Image.fromarray(blank_tile,"RGB")
        draw_tile = ImageDraw.Draw(blank_tile_as_img) 
        draw_polygons(draw_tile, polygons)
        rgb_array = np.array(blank_tile_as_img).astype(np.int32)
        #rgb_array = rgb_array.astype(np.uint32)
        int_array = np.zeros((pred_width, pred_height), dtype=np.int32)
        int_array = ((rgb_array[:,:,0] << 16) | (rgb_array[:,:,1] << 8) | rgb_array[:,:,2])
        print("int_array height:", int_array.shape[0])
        print("int_array width:", int_array.shape[1])
        currentMax = np.max(int_array)
        print(currentMax)
        if (currentMax > maxValue): maxValue = currentMax
        
        # need a new version of image_array where we decode the 3 channels into 1 channel
        # ok I see the issue, we can only draw the polygons on the PIL image, and I don't know a way to draw them with different values
        
        # big_array[i:i - start_y, j:j - start_x] = image_array
        # try:
            #if (int_array[0] > tile.shape[0] or int_array[1] > tile.shape[1]):
        big_array[j:j + tile.shape[1], i:i + tile.shape[0]] = int_array
        # except:
        #     print("ERRRORORORORORORORO")
        #     pass
        # big_array[i-start_x:i-start_x+tile.shape[0] , j-start_y:j - start_y+tile.shape[1]] = int_array
        
        print("iteration: ", iteration)
        
print("biggest value in big_array:", np.max(big_array))